In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import json


In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [13]:
table = soup.find_all('table', class_='wikitable')[0]

In [14]:
films_html = table.find_all('tr')[1:]

In [15]:
films = []
for film_html in films_html:
    attributes = film_html.find_all('td')
    film_title = film_html.find("th").find("a").text.strip()
    film_ref = film_html.find("th").find("a")["href"]
    year = attributes[3].text.strip()
    revenue_str = attributes[2].text.strip()
    revenue = [ch for ch in revenue_str if ch.isdigit()]
    revenue = int("".join(revenue))
    films.append([film_title, year, revenue, film_ref])

In [16]:
def get_country_and_director(href):
    response = requests.get("https://en.wikipedia.org" + href)
    soup = BeautifulSoup(response.text, "html.parser")
    data = soup.select_one("table", class_="infobox")
    country = data.find("th", string="Country")
    if not country:
        country_td = data.find("th", string="Countries").find_next("td")
        ul = country_td.find("ul")
        country = (", ".join([li.text.strip() for li in ul.find_all("li")]) if ul else country_td.text.strip()
        )
    else:
        country = country.find_next("td").text.strip()

    director_td = data.find("th", string="Directed by").find_next("td")
    directors = director_td.find("ul")
    director = (", ".join([li.text.strip() for li in directors.find_all("li")]) if directors else director_td.text.strip()
    )
    return country, director

In [17]:
cleaned_films = []
for i, film in enumerate(films):
    cleaned_film = film[:3]
    try:
        
        country, director = get_country_and_director(film[3])
        cleaned_films.append(cleaned_film + [country, director])
    except Exception as e:
        print(f"Error getting country and director for {film[0]}: {e}")
        cleaned_films.append(cleaned_film + ["N/A", "N/A"])
        

Error getting country and director for Ne Zha 2: 'NoneType' object has no attribute 'find_next'


In [18]:
# Create connection to database


conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Create the films table
cursor.execute('''
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office REAL,
    country TEXT
)
''')

# Insert data from cleaned_films
# cleaned_films structure is [title, year, revenue, country, director]
for film in cleaned_films:
    cursor.execute('''
    INSERT INTO films (title, release_year, box_office, country, director)
    VALUES (?, ?, ?, ?, ?)
    ''', (film[0], int(film[1]), film[2], film[3], film[4]))

# Commit changes and close connection
conn.commit()
conn.close()